In [146]:
import csv
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime


In [121]:
import sqlite3
from sqlite3 import Error
 
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    
    return conn

In [122]:
conn=create_connection("C:\db\claim.db")


2.6.0


In [123]:
os.chdir('C:\\db')

In [125]:

with open ('insurance_claims.csv', 'r') as f:
    cursor = conn.cursor()
    reader = csv.reader(f)
    columns = next(reader) 
    #query = " CREATE TABLE MyTable ({0}) "
    #query = query.format(', '.join(columns))
    #query=query.replace('-','_')
    #print(query)
    #cursor.execute(query)

    query = " insert into MyTable({0}) values ({1}) "
    query = query.format(', '.join(columns), ','.join('?' * (len(columns))))
    for data in reader:
        x=data
        query=query.replace('-','_')
        cursor.execute(query, x)
    conn.commit()
    conn.close()

    
    

    

In [126]:
conn=create_connection("C:\db\claim.db")


2.6.0


In [133]:
numl=[]
strl=[]
for n in range(48,58):
    numl.append(chr(n))
for n in range(65,127):
    strl.append(chr(n))


In [128]:
df = pd.read_sql_query("select * from MyTable ;", conn, coerce_float =True)
df= df.replace('?',np.nan)
df= df.replace(',','.')
df= df.drop(columns=['policy_csl'])
l= list(df)


for col in l:
    if any(t in df[col].loc[0] for t in strl ):
        df[col]=df[col].astype('str')
    else: 
        if '-' in df[col].loc[0]:
            df[col]=pd.to_datetime(df[col])

        else:
            df[col]=df[col].astype('float')

conn.close()       

In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 38 columns):
months_as_customer             1000 non-null float64
age                            1000 non-null float64
policy_number                  1000 non-null float64
policy_bind_date               1000 non-null datetime64[ns]
policy_state                   1000 non-null object
policy_deductable              1000 non-null float64
policy_annual_premium          1000 non-null float64
umbrella_limit                 1000 non-null float64
insured_zip                    1000 non-null float64
insured_sex                    1000 non-null object
insured_education_level        1000 non-null object
insured_occupation             1000 non-null object
insured_hobbies                1000 non-null object
insured_relationship           1000 non-null object
capital_gains                  1000 non-null float64
capital_loss                   1000 non-null float64
incident_date                  1000 non-null 

In [118]:
import pandas_profiling as pp
from IPython.core.display import display, HTML
display(HTML(pp.ProfileReport(df).to_html()))



Number of variables,38
Number of observations,2000
Total Missing (%),0.0%
Total size in memory,593.8 KiB
Average record size in memory,304.0 B
Numeric,16
Categorical,18
Boolean,0
Date,2
Text (Unique),0
Rejected,2


In [132]:
pp.ProfileReport(df).to_file('insuance_claims.html')

In [159]:
#change year for policy_bind_date for test
df['policy_bind_date']=df['policy_bind_date'].apply(lambda dt: dt.replace(year=2016))
df['incident_date']=df['incident_date'].apply(lambda dt: dt.replace(year=2016))


In [166]:
#Plan statistique T1 2016
t1_2016=df.set_index('policy_bind_date')['2016-01-01':'2016-03-31']
Primes_Souscrites = t1_2016['policy_annual_premium'].sum()
Primes_Souscrites_Moyenne = t1_2016['policy_annual_premium'].mean()
Nombre_de_sinistres = t1_2016['vehicle_claim'].count()
Sinistres= t1_2016['vehicle_claim'].sum()

print('Primes Souscrites : ' + str(Primes_Souscrites))
print('Primes Souscrites Moyenne : '+ str(Primes_Souscrites_Moyenne) )
print('Nombre de sinistres : ' +str(Nombre_de_sinistres))
print('Sinistres : ' +str(Sinistres))
print('Sinistres moyen : ' + str(Sinistres/Nombre_de_sinistres))                          
print('Rapport de Sinistres à Primes % : ' + str(Sinistres/Primes_Souscrites*100) ) 



Primes Souscrites : 315005.47
Primes Souscrites Moyenne : 1250.021706349206
Nombre de sinistres : 252
Sinistres : 9304770.0
Sinistres moyen : 36923.69047619047
Rapport de Sinistres à Primes % : 2953.8439443607126
